In [2]:
-- Library Management System
-- Author: Divyam
-- Description: A SQL project to simulate a library's database for managing books, members, and borrowing activities.

-- Project Overview:
-- The Library Management System is designed to streamline the operations of a library.
-- It includes functionalities to manage books, track members, and record borrowing activities.
-- This project uses SQL to create a relational database with features such as data integrity, automation through triggers, and insightful queries to derive meaningful reports.
-- The goal is to provide an efficient way to handle library operations and analyze data for decision-making.

-- Key Objectives:
-- 1. Create a structured database with tables for books, members, and borrowing activities.
-- 2. Implement data integrity using constraints and relationships between tables.
-- 3. Automate routine tasks like updating book availability using triggers.
-- 4. Provide useful queries to gain insights, such as the most borrowed books and overdue records.
-- 5. Export data for external analysis and reporting.

-- Step 1: Create the Database
CREATE DATABASE LibraryDB;
\c LibraryDB;  -- Connect to the newly created database

-- Step 2: Create Tables with Constraints

-- Books Table: Stores information about books in the library.
CREATE TABLE Books (
    BookID SERIAL PRIMARY KEY,           -- Unique identifier for each book.
    Title VARCHAR(100) NOT NULL,         -- Title of the book (cannot be null).
    Author VARCHAR(100),                 -- Author of the book.
    Genre VARCHAR(50),                   -- Genre of the book.
    Copies INT CHECK (Copies >= 0)       -- Number of copies available (must be >= 0).
);

-- Members Table: Stores information about library members.
CREATE TABLE Members (
    MemberID SERIAL PRIMARY KEY,         -- Unique identifier for each member.
    Name VARCHAR(100) NOT NULL,          -- Name of the member (cannot be null).
    Email VARCHAR(100) UNIQUE NOT NULL,  -- Email address (must be unique).
    JoinDate DATE NOT NULL               -- Date the member joined the library.
);

-- Borrowed_Books Table: Records the books borrowed by members.
CREATE TABLE Borrowed_Books (
    BorrowID SERIAL PRIMARY KEY,         -- Unique identifier for each borrowing transaction.
    MemberID INT,                        -- Foreign key referencing the member who borrowed the book.
    BookID INT,                          -- Foreign key referencing the book borrowed.
    BorrowDate DATE NOT NULL,            -- Date the book was borrowed.
    ReturnDate DATE,                     -- Date the book was returned (can be null if not returned).
    FOREIGN KEY (MemberID) REFERENCES Members(MemberID),  -- Foreign key constraint on MemberID.
    FOREIGN KEY (BookID) REFERENCES Books(BookID)         -- Foreign key constraint on BookID.
);

-- Step 3: Insert Sample Data
-- Explanation: Sample data is added to the tables to simulate real-world scenarios, allowing us to test the database functionality effectively.

-- Insert Books: Adding a few books to the library.
INSERT INTO Books (Title, Author, Genre, Copies)
VALUES
('1984', 'George Orwell', 'Dystopian', 5),
('To Kill a Mockingbird', 'Harper Lee', 'Fiction', 3),
('The Great Gatsby', 'F. Scott Fitzgerald', 'Classic', 4);

-- Insert Members: Adding a few members to the library.
INSERT INTO Members (Name, Email, JoinDate)
VALUES
('Alice Johnson', 'alice@example.com', '2024-01-10'),
('Bob Smith', 'bob@example.com', '2024-02-15');

-- Insert Borrowed_Books: Adding a few borrowing records.
INSERT INTO Borrowed_Books (MemberID, BookID, BorrowDate, ReturnDate)
VALUES
(1, 1, '2024-12-20', NULL),
(2, 2, '2024-12-22', '2024-12-25');

-- Step 4: Create Triggers for Automation
-- Explanation: Triggers are used to automate the update of book availability when borrowing or returning occurs, ensuring accurate data management.

-- Trigger to Decrease Copies When a Book is Borrowed
CREATE OR REPLACE FUNCTION DecreaseCopies() RETURNS TRIGGER AS $$
BEGIN
    UPDATE Books
    SET Copies = Copies - 1
    WHERE BookID = NEW.BookID AND Copies > 0;  -- Only decrease if copies > 0
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Create the Trigger for DecreaseCopies function.
CREATE TRIGGER DecreaseCopiesTrigger
AFTER INSERT ON Borrowed_Books
FOR EACH ROW
EXECUTE FUNCTION DecreaseCopies();

-- Trigger to Increase Copies When a Book is Returned
CREATE OR REPLACE FUNCTION IncreaseCopies() RETURNS TRIGGER AS $$
BEGIN
    IF NEW.ReturnDate IS NOT NULL THEN
        UPDATE Books
        SET Copies = Copies + 1
        WHERE BookID = NEW.BookID;  -- Increase copies when book is returned
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Create the Trigger for IncreaseCopies function.
CREATE TRIGGER IncreaseCopiesTrigger
AFTER UPDATE ON Borrowed_Books
FOR EACH ROW
EXECUTE FUNCTION IncreaseCopies();

-- Step 5: Useful Queries
-- Explanation: These queries are designed to retrieve actionable insights from the database, such as viewing all books, listing borrowed books, and identifying overdue records.

-- View All Books: Displays all books in the library.
SELECT * FROM Books;

-- List Members Who Borrowed Books: Shows which members have borrowed which books.
SELECT Members.Name, Books.Title, Borrowed_Books.BorrowDate, Borrowed_Books.ReturnDate
FROM Borrowed_Books
JOIN Members ON Borrowed_Books.MemberID = Members.MemberID
JOIN Books ON Borrowed_Books.BookID = Books.BookID;

-- Check Available Copies of a Book: Shows the number of available copies of a specific book.
SELECT Title, Copies
FROM Books
WHERE Title = '1984';

-- Add a New Borrowing Record: Adds a new record when a member borrows a book.
INSERT INTO Borrowed_Books (MemberID, BookID, BorrowDate, ReturnDate)
VALUES (1, 3, '2024-12-26', NULL);

-- Update Return Date for a Borrowed Book: Updates the return date for a borrowed book.
UPDATE Borrowed_Books
SET ReturnDate = '2024-12-30'
WHERE BorrowID = 1;

-- Delete a Member: Deletes a member from the system.
DELETE FROM Members
WHERE MemberID = 2;

-- Find the Most Borrowed Book: Identifies the book that has been borrowed the most.
SELECT Books.Title, COUNT(Borrowed_Books.BookID) AS BorrowCount
FROM Borrowed_Books
JOIN Books ON Borrowed_Books.BookID = Books.BookID
GROUP BY Books.Title
ORDER BY BorrowCount DESC
LIMIT 1;

-- List Overdue Books: Displays books that have been borrowed but not returned within the last 14 days.
SELECT Members.Name, Books.Title, Borrowed_Books.BorrowDate, Borrowed_Books.ReturnDate
FROM Borrowed_Books
JOIN Members ON Borrowed_Books.MemberID = Members.MemberID
JOIN Books ON Borrowed_Books.BookID = Books.BookID
WHERE Borrowed_Books.ReturnDate IS NULL AND Borrowed_Books.BorrowDate < CURRENT_DATE - INTERVAL '14 days';

Error: -- Library Management System
-- Author: Divyam
-- Description: A SQL project to simulate a library's database for managing books, members, and borrowing activities.

-- Project Overview:
-- The Library Management System is designed to streamline the operations of a library.
-- It includes functionalities to manage books, track members, and record borrowing activities.
-- This project uses SQL to create a relational database with features such as data integrity, automation through triggers, and insightful queries to derive meaningful reports.
-- The goal is to provide an efficient way to handle library operations and analyze data for decision-making.

-- Key Objectives:
-- 1. Create a structured database with tables for books, members, and borrowing activities.
-- 2. Implement data integrity using constraints and relationships between tables.
-- 3. Automate routine tasks like updating book availability using triggers.
-- 4. Provide useful queries to gain insights, such as the mos